<a href="https://colab.research.google.com/github/pagiorgi/pellusitas/blob/main/cleanImagepelusa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Clonar el repo


In [1]:
!git clone https://github.com/pagiorgi/pellusitas.git
%cd pellusitas


Cloning into 'pellusitas'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 384.57 KiB | 14.24 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/pellusitas


## quitar el fondo

In [2]:
import argparse
import cv2
import numpy as np
import sys

In [3]:
def remove_grid_by_color(img_bgr, lower_hsv=(80, 30, 40), upper_hsv=(140, 255, 255)):
    """Detecta y elimina líneas/grilla de color (azul/cian) usando enmascarado en HSV + inpainting.
    Los valores HSV por defecto están pensados para líneas azules/cian en papel blanco; ajusta si es necesario.
    Devuelve la imagen reconstruida (BGR) y la máscara usada para inpainting.
    """
    img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower = np.array(lower_hsv, dtype=np.uint8)
    upper = np.array(upper_hsv, dtype=np.uint8)
    mask = cv2.inRange(img_hsv, lower, upper)

    # Refina la máscara: elimina ruido y conecta trazos de línea
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)

    # Aumentar un poco el grosor de la máscara para cubrir mejor las líneas al inpaint
    k2 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.dilate(mask, k2, iterations=1)

    # Inpaint (reconstruye zonas donde estaba la grilla)
    inpainted = cv2.inpaint(img_bgr, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
    return inpainted, mask


def remove_dark_spots(img_bgr, spot_area_threshold=150):
    """Detecta pequeñas manchas oscuras y las elimina usando inpainting.
    spot_area_threshold: área mínima (en pixeles) para que un componente se considere una mancha a eliminar.
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # Realizamos un umbral para detectar marcas oscuras
    _, th = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV)

    # Limpiar el umbral y obtener contornos de las manchas
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations=1)

    # Buscar componentes conectados
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, connectivity=8)
    mask = np.zeros_like(gray, dtype=np.uint8)
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area <= spot_area_threshold:
            # agrega al mask para inpaint
            mask[labels == i] = 255

    if mask.sum() == 0:
        return img_bgr, mask

    # Suaviza la máscara un poco
    mask = cv2.dilate(mask, kernel, iterations=1)
    clean = cv2.inpaint(img_bgr, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
    return clean, mask


def enhance_and_denoise(img_bgr):
    """Aplica filtros de suavizado y contrasta ligeramente la imagen para un aspecto más limpio."""
    # Denoise / suavizado
    den = cv2.bilateralFilter(img_bgr, d=9, sigmaColor=75, sigmaSpace=75)

    # Aumentar ligeramente el contraste usando CLAHE en cada canal
    lab = cv2.cvtColor(den, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    lab2 = cv2.merge((l2, a, b))
    out = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)
    return out


def make_transparent_background(img_bgr, bg_threshold=240):
    """Convierte el fondo blanco/casi blanco en transparente generando un canal alpha.
    bg_threshold: intensidad mínima para considerar píxel como fondo (0-255)
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # Fondo: pixeles con intensidad alta
    _, alpha = cv2.threshold(gray, bg_threshold, 255, cv2.THRESH_BINARY)

    # Invertir para que alpha=255 sea donde hay contenido
    alpha = 255 - alpha

    # Suavizar el canal alpha para bordes más agradables
    alpha = cv2.GaussianBlur(alpha, (5, 5), 0)

    b, g, r = cv2.split(img_bgr)
    rgba = cv2.merge((b, g, r, alpha))
    return rgba


def parse_args():
    p = argparse.ArgumentParser(description='Limpiar imagen: quitar fondo cuadriculado y pequeñas imperfecciones')
    p.add_argument('--input', '-i', required=True, help='Ruta imagen entrada (ej: /mnt/data/pelusa.jpeg)')
    p.add_argument('--out_clean', default='/mnt/data/pelusa_clean.png', help='Ruta imagen limpia salida (png)')
    p.add_argument('--out_transparent', default='/mnt/data/pelusa_transparent.png', help='Ruta imagen con fondo transparente (png)')
    p.add_argument('--debug_masks', action='store_true', help='Guardar máscaras intermedias para depuración')
    return p.parse_args()



In [4]:
# ==== Parámetros de entrada/salida ====
input_path = "pelusa.jpeg"
out_clean = "pelusa_clean.png"
out_transparent = "pelusa_transparent.png"
debug_masks = True

In [6]:
# ==== Proceso ====
img = cv2.imread(input_path)
if img is None:
    raise FileNotFoundError(f"No se pudo leer la imagen {input_path}")


# Paso 1: eliminar la grilla (líneas azules/cian)
inpainted_grid, mask_grid = remove_grid_by_color(img)


# Paso 2: eliminar manchas oscuras pequeñas e imperfecciones
inpainted_spots, mask_spots = remove_dark_spots(inpainted_grid)


# Paso 3: mejora y suavizado final
enhanced = enhance_and_denoise(inpainted_spots)


# Guardar resultado limpio
cv2.imwrite(out_clean, enhanced)
print(f"Guardada imagen limpia en: {out_clean}")


# Paso 4: generar versión con fondo transparente
rgba = make_transparent_background(enhanced)
cv2.imwrite(out_transparent, rgba)
print(f"Guardada imagen con fondo transparente en: {out_transparent}")


if debug_masks:
   cv2.imwrite('/_mask_grid.png', mask_grid)
   cv2.imwrite('/_mask_spots.png', mask_spots)
   print('Guardadas máscaras de depuración: /_mask_grid.png, /_mask_spots.png')

Guardada imagen limpia en: pelusa_clean.png
Guardada imagen con fondo transparente en: pelusa_transparent.png
Guardadas máscaras de depuración: /mnt/data/_mask_grid.png, /mnt/data/_mask_spots.png
